# Bert 

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Download Dataset

In [ ]:
# Download link 1
#!gdown --id '1znKmX08v9Fygp-dgwo7BKiLIf2qL1FH1' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
#!gdown --id '1pOu3FdPdvzielUZyggeD7KDnVy9iW1uC' --output hw7_data.zip

#!unzip -o hw7_data.zip

#都沒辦法下載 直接載下來放在資料夾裡面

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Tue May  2 08:22:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [ ]:
# You are allowed to change version of transformers or use other toolkits

!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Packages

In [ ]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [ ]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

## Load Model and Tokenizer

A list of avaliable pre-trained models:https://huggingface.co/models




 

In [ ]:
#先將Pre-train模型導入 這邊使用bert-base-chinese

model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")
# eng_tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

## [範例] Tokenize 

In [ ]:
# 因套入的是中文tokenizer，這邊就用中文
chi_paragraph = '李弘毅大讚，2023 good'
tokens = tokenizer.tokenize(chi_paragraph)
print(tokens)

#one-hot vector
tokenizer.convert_tokens_to_ids(tokens)

['李', '弘', '毅', '大', '讚', '，', '202', '##3', 'good']


[3330, 2473, 3675, 1920, 6367, 8024, 9707, 8152, 9005]

## [範例] Encoder vs Decode

In [ ]:
question = '李弘毅到底幾班?'
paragraph = '李弘毅幾班大金'
encoded = tokenizer.encode(question,paragraph)
decoded = tokenizer.decode(encoded)
print(encoded)
print(decoded)
# 可以看到 開頭cls 都是token = 101 結束sep token=102

[101, 3330, 2473, 3675, 1168, 2419, 2407, 4408, 136, 102, 3330, 2473, 3675, 2407, 4408, 1920, 7032, 102]
[CLS] 李 弘 毅 到 底 幾 班? [SEP] 李 弘 毅 幾 班 大 金 [SEP]


## [範例] Model Inputs

In [ ]:
inputs_sample = tokenizer(question, paragraph, return_tensors='pt')
# Indices of input sequence tokens in the vocabulary
print('Input ids:    ', inputs_sample['input_ids'])

# Segment token indices to indicate first and second pertrain of the inputs
# Indices are selected in [0, 1]
#把文字分類是問題還是文章
print('Token type ids: ', inputs_sample['token_type_ids'])

# Mask to avoid preforming attention on padding tokens indices.
# Mask values selected in [0, 1]
print('Attention mask: ', inputs_sample['attention_mask'])
# 如果同時輸入的是多個seq時，長度不一樣的話，模型會直接補0
# 範例:
print('\n\n')
print('-------------------範例-------------------')
print('\n\n')
ss = ['今天真是個好天氣', #8個字
    '那麼明天呢? 我不太確定'] #11個字
inputs = tokenizer(ss, padding=True, return_tensors='pt')
# 可以發現它們差了3個字，所以第一句的token後面會補0
# 那我們要讓模型知道後面那三個token是沒有意義的，attention_mask 因此這邊可以看到它們的attention都是0
print(inputs)

Input ids:     tensor([[ 101, 3330, 2473, 3675, 1168, 2419, 2407, 4408,  136,  102, 3330, 2473,
         3675, 2407, 4408, 1920, 7032,  102]])
Token type ids:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]])
Attention mask:  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])



-------------------範例-------------------



{'input_ids': tensor([[ 101,  791, 1921, 4696, 3221,  943, 1962, 1921, 3706,  102,    0,    0,
            0],
        [ 101, 6929, 7938, 3209, 1921, 1450,  136, 2769,  679, 1922, 4825, 2137,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


## [範例] Model testing


In [ ]:
question = '李弘毅幾班?'
paragraph = '李弘毅幾班大金。'
inputs_sample = tokenizer(question, paragraph, return_tensors='pt')

with torch.no_grad():
  output_sample = model(**inputs_sample)

# output = model(inputs_sample['input_ids'], inputs_sample['token_type_ids'], inputs_sample['attention_mask'])

print('start_logits: ')
print(output_sample.start_logits)

print('end_logits: ')
print(output_sample.end_logits)

# 找出分數最高的 就是開始跟結束的位置
start = torch.argmax(output_sample.start_logits)
end = torch.argmax(output_sample.end_logits)
print('start position: ', start.item())
print('end position: ', end.item())


# 然後把位置抓出來
predict_id = inputs_sample['input_ids'][0][start : end + 1]
print('predict_id: ', predict_id)

predict_answer = tokenizer.decode(predict_id)
print('predict_answer: ', predict_answer)

start_logits: 
tensor([[-0.1764,  0.2807,  0.6335,  0.3332,  0.3117,  0.5854,  0.0033, -0.1607,
          0.4784,  0.3804,  0.1173,  0.2362,  0.3875, -0.2558, -0.0157, -0.3988,
         -0.1607]])
end_logits: 
tensor([[ 0.7969, -0.1008, -1.0536, -0.6032, -0.7722, -0.5142,  0.2136,  0.4946,
         -0.9369, -0.9783, -0.6090, -0.4152, -0.5355, -0.0907, -0.0044,  0.0238,
          0.4946]])
start position:  2
end position:  0
predict_id:  tensor([], dtype=torch.int64)
predict_answer:  


## [範例] 我們訓練一下模型

In [ ]:
output_after_train = model(**inputs_sample, start_positions = torch.tensor([13]), end_positions = torch.tensor([14]))
print('loss: ', output_after_train.loss)

opt = AdamW(model.parameters(), lr = 1e-4)
output_after_train.loss.backward()
opt.step()

loss:  tensor(3.0016, grad_fn=<DivBackward0>)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## [範例] 再將我們模型重跑一次結果
--> 就可得到 大金這個答案!

In [ ]:
question = '李弘毅幾班?'
paragraph = '李弘毅幾班大金。'
inputs_sample = tokenizer(question, paragraph, return_tensors='pt')

with torch.no_grad():
  output_sample = model(**inputs_sample)

# output = model(inputs_sample['input_ids'], inputs_sample['token_type_ids'], inputs_sample['attention_mask'])

print('start_logits: ')
print(output_sample.start_logits)

print('end_logits: ')
print(output_sample.end_logits)

# 找出分數最高的 就是開始跟結束的位置
start = torch.argmax(output_sample.start_logits)
end = torch.argmax(output_sample.end_logits)
print('start position: ', start.item())
print('end position: ', end.item())


# 然後把位置抓出來
predict_id = inputs_sample['input_ids'][0][start : end + 1]
print('predict_id: ', predict_id)

predict_answer = tokenizer.decode(predict_id)
print('predict_answer: ', predict_answer)

start_logits: 
tensor([[-0.4422, -0.5756, -0.8748, -1.4876, -0.5132, -0.1957,  0.1331, -1.2201,
         -0.6473, -1.0608, -1.5258, -0.6746, -0.4063,  1.4856,  0.3126, -0.1252,
         -1.2201]])
end_logits: 
tensor([[-0.6428, -1.5354, -2.5451, -2.2400, -1.5198, -0.8230, -1.0263, -1.8690,
         -2.1773, -2.5474, -2.1660, -1.2789, -0.9753, -0.2520,  1.0890, -0.3671,
         -1.8690]])
start position:  13
end position:  14
predict_id:  tensor([1920, 7032])
predict_answer:  大 金


## Read Data

- Training set: 26935 QA pairs
- Dev set: 3523  QA pairs
- Test set: 3492  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("/content/drive/MyDrive/NTU_ML_HW/HW7/hw7_train.json")
dev_questions, dev_paragraphs = read_data("/content/drive/MyDrive/NTU_ML_HW/HW7/hw7_dev.json")
test_questions, test_paragraphs = read_data("/content/drive/MyDrive/NTU_ML_HW/HW7/hw7_test.json")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("Original_train_questions",train_questions[0])

Original_train_questions {'id': 0, 'paragraph_id': 3884, 'question_text': '羅馬教皇利奧三世在800年正式加冕誰為羅馬人的皇帝?', 'answer_text': '查理大帝', 'answer_start': 141, 'answer_end': 144}


## Tokenize Data

In [ ]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print("train_questions_tokenized",train_questions_tokenized)  #變成一堆向量了

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Dataset and Dataloader

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 150
        
        ##### TODO: Change value of doc_stride ##### 
        #滑窗法切分超長文檔中用到的步長 （這個值不要設置的太小！！因為是取min，會擠爆你的內存！！血的教訓）
        
        #就是window size拉，因為文章太長，bert又限制512長度下，我們直接切一段一段來判斷有沒有答案
        #每150個token當成一段
        #self.doc_stride = 150

        #這邊改大一點，以免答案剛好在window的邊緣，而被切掉。
        self.doc_stride = 300

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]  # BERT常用512
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 16

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [ ]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

## Training

In [ ]:
from transformers import get_linear_schedule_with_warmup #做linear learning rate decay

num_epoch = 2 
validation = True
logging_step = 100
learning_rate = 1e-4
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps = 0,num_training_steps = total_steps)

if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader, scheduler) 

model.train()

print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for data in tqdm(train_loader):	
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        if fp16_training:
            accelerator.backward(output.loss)
        else:
            output.loss.backward()
        
        #更新參數
        optimizer.step()
        optimizer.zero_grad()


        ##### TODO: Apply linear learning rate decay #####
        #更新學習率
        scheduler.step()



        step += 1

        
        # Print training loss and accuracy over past logging step
        # 每經過100次跌代(iteration)，就輸出進度訊息
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/1684 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 1.849, acc = 0.391
Epoch 1 | Step 200 | loss = 0.966, acc = 0.623
Epoch 1 | Step 300 | loss = 0.805, acc = 0.669
Epoch 1 | Step 400 | loss = 0.846, acc = 0.660
Epoch 1 | Step 500 | loss = 0.767, acc = 0.686
Epoch 1 | Step 600 | loss = 0.739, acc = 0.686
Epoch 1 | Step 700 | loss = 0.654, acc = 0.727
Epoch 1 | Step 800 | loss = 0.620, acc = 0.736
Epoch 1 | Step 900 | loss = 0.626, acc = 0.732
Epoch 1 | Step 1000 | loss = 0.547, acc = 0.752
Epoch 1 | Step 1100 | loss = 0.616, acc = 0.733
Epoch 1 | Step 1200 | loss = 0.549, acc = 0.751
Epoch 1 | Step 1300 | loss = 0.542, acc = 0.748
Epoch 1 | Step 1400 | loss = 0.534, acc = 0.746
Epoch 1 | Step 1500 | loss = 0.504, acc = 0.772
Epoch 1 | Step 1600 | loss = 0.499, acc = 0.772
Evaluating Dev Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.511


  0%|          | 0/1684 [00:00<?, ?it/s]

Epoch 2 | Step 100 | loss = 0.280, acc = 0.841
Epoch 2 | Step 200 | loss = 0.301, acc = 0.865
Epoch 2 | Step 300 | loss = 0.252, acc = 0.863
Epoch 2 | Step 400 | loss = 0.281, acc = 0.857
Epoch 2 | Step 500 | loss = 0.281, acc = 0.846
Epoch 2 | Step 600 | loss = 0.252, acc = 0.871
Epoch 2 | Step 700 | loss = 0.234, acc = 0.875
Epoch 2 | Step 800 | loss = 0.222, acc = 0.883
Epoch 2 | Step 900 | loss = 0.246, acc = 0.870
Epoch 2 | Step 1000 | loss = 0.244, acc = 0.873
Epoch 2 | Step 1100 | loss = 0.249, acc = 0.881
Epoch 2 | Step 1200 | loss = 0.240, acc = 0.870
Epoch 2 | Step 1300 | loss = 0.213, acc = 0.888
Epoch 2 | Step 1400 | loss = 0.222, acc = 0.878
Epoch 2 | Step 1500 | loss = 0.227, acc = 0.881
Epoch 2 | Step 1600 | loss = 0.215, acc = 0.877
Evaluating Dev Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Validation | Epoch 2 | acc = 0.549
Saving Model ...


## Testing

In [ ]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3493 [00:00<?, ?it/s]

Completed! Result is in result.csv


In [ ]:
test_question

{'id': 3492,
 'paragraph_id': 850,
 'question_text': '《瘟疫論》這本書在中國醫學方面的意義是?',
 'answer_text': None,
 'answer_start': None,
 'answer_end': None}

In [ ]:
result[3492]

'溫病學派'